Primeiro, carregamos as bibliotecas aplicáveis: o pandas para manipulação de dados, o openai para a API de geração de texto e o json para manipulação de dados JSON. O PyPDF2 é usado para ler o PDF e o dotenv para carregar as variáveis de ambiente.

In [3]:
import pandas as pd
import requests
from openai import OpenAI
import os
import PyPDF2
from dotenv import load_dotenv
import json
load_dotenv()

True

O arquivo .env contém a chave da API do OpenAI, que é carregada usando a biblioteca dotenv. Esse arquivo não está no repositório, mas você pode criar um arquivo .env com a chave da API do OpenAI, com o nome da variável `OPENAI_API_KEY`.

Abaixo, baixamos o arquivo do TJSP usando a biblioteca requests e o salvamos em um arquivo local. Em seguida, usamos o PyPDF2 para ler o arquivo e extrair o texto.

In [4]:
u = 'https://www.tjsp.jus.br/Download/Portal/Biblioteca/Biblioteca/Curriculum/Curriculum.pdf'
r = requests.get(u)
with open('Curriculum.pdf', 'wb') as f:
  f.write(r.content)

Esta função lê o PDF e retorna uma lista de strings, onde cada string é uma página do PDF.

In [2]:
def pdf_to_text_pages(pdf_file):
  # Open the PDF file in read-binary mode
  with open(pdf_file, 'rb') as file:
    # Create a PDF file reader object
    pdf_reader = PyPDF2.PdfReader(file)

    # Initialize an empty list to store the text of each page
    text_pages = []
    # Loop over the number of pages in the PDF file
    for page_num in range(len(pdf_reader.pages)):
      # Extract the text from each page and append it to the list
      page = pdf_reader.pages[page_num]
      text_pages.append(page.extract_text())

  return text_pages

# Usage
pdf_file = 'Curriculum.pdf'

text_pages = pdf_to_text_pages(pdf_file)


FileNotFoundError: [Errno 2] No such file or directory: 'https://www.tjsp.jus.br/Download/Portal/Biblioteca/Biblioteca/Curriculum/Curriculum.pdf'

In [5]:
print(text_pages[-1])

FRANCISCO CARLOS INOUYE SHINTATE  
Nascido em Valparaíso , São Paulo, em 07.07 .1967 . Bacharel em 
Direito pela Faculdade de Direito da Universidade de São Paulo , em 
1989. Promotor Público do Ministério Públic o do Estado de São 
Paulo, de 31.10.1990  a 10.01.1991. Ingressou na Magistratura em 
11.01.1991 . Atuou em Sorocaba, Potirendaba ( São José do Rio Preto ), 
Tatuí e Capital. Juiz Substituto de 2ª Instância em 12.12 .2019 . 
Nome ado Desembargador do Tribunal de Justiça de São Paulo, em 
07.12 .2023.  
 



Agora, carregamos o motor GPT-4 da OpenAI

In [5]:
client = OpenAI()

O prompt abaixo é a instrução inicial passada ao modelo para que ele saiba o que estamos tentando fazer. Em seguida, passaremos o texto de cada página do PDF para o modelo, que retornará os dados estruturados para cada página.

In [1]:
# Load the prompt from the file
with open('prompt.md', 'r') as file:
  prompt = file.read()

print(prompt)

Você é um assistente de IA que está me ajudando a extrair dados de desembargadores do TJSP para um projeto de pesquisa. Você receberá as informações de um desembargador em linguagem natural e retornará um JSON com as informações processadas. O JSON deve conter as seguintes informações:


{
  "nome": "nome completo do desembargador",
  "data_nascimento": "YYYY-MM-DD",
  "municipio_nascimento": "município de nascimento",
  "estado_nascimento": "estado de nascimento",
  "faculdade_direito": "nome da faculdade de direito",
  "ano_formatura": "ano de formatura na faculdade de direito",
  "data_magistratura": "YYYY-MM-DD (data de ingresso na magsitratura)",
  "juiz_substituto_2grau": "sim ou não",
  "data_juiz_substituto_2grau": "YYYY-MM-DD (data de promoção para juiz substituto de 2o grau)",
  "data_promocao_desembargador": "YYYY-MM-DD (data de promoção para desembargador)",
  "data_aposentadoria": "YYYY-MM-DD (data de aposentadoria)",
  "data_falecimento": "YYYY-MM-DD (data de faleci

A função abaixo é usada para enviar uma solicitação para a API do OpenAI e retornar os dados estruturados. O resultado é salvo em um arquivo JSON, para que possamos acessá-lo posteriormente caso a API fique fora do ar. Os arquivos brutos foram colocados nos [releases]() do repositório.

In [ ]:

def pegar_texto(i_pag, text_pages):
  # if file does not exist
  file_name = f'json/desembargadores/pag_{str(i_pag).zfill(4)}.json'
  if not os.path.exists(file_name):
    completion = client.chat.completions.create(
      model="gpt-4-turbo-preview",
      messages=[
        {
          "role": "system",
          "content": prompt
        },
        {
          "role": "user",
          "content": text_pages[i_pag]
        }
      ],
      temperature=0,
      response_format={"type": "json_object"}
    )
    txt = completion.choices[0].message.content
    with open(file_name, 'a') as file:
      file.write(txt)
  else:
    with open(file_name, 'r') as file:
      txt = file.read()
  result = json.loads(txt)
  result['page'] = i_pag+1
  return result


Aqui, rodamos a função de solicitação para cada página do PDF e salvamos os resultados em arquivos JSON. O resultado é uma lista de arquivos JSON, onde cada arquivo contém os dados estruturados de uma página do PDF.

In [8]:
lista_desemb = []

for i in range(len(text_pages)):
  if i % 50 == 0:
    print("Pegando página: " + str(i+1) + " de " + str(len(text_pages)) + "...")
  lista_desemb.append(pegar_texto(i, text_pages))


Pegando página: 1 de 1351...
Pegando página: 51 de 1351...
Pegando página: 101 de 1351...
Pegando página: 151 de 1351...
Pegando página: 201 de 1351...
Pegando página: 251 de 1351...
Pegando página: 301 de 1351...
Pegando página: 351 de 1351...
Pegando página: 401 de 1351...
Pegando página: 451 de 1351...
Pegando página: 501 de 1351...
Pegando página: 551 de 1351...
Pegando página: 601 de 1351...
Pegando página: 651 de 1351...
Pegando página: 701 de 1351...
Pegando página: 751 de 1351...
Pegando página: 801 de 1351...
Pegando página: 851 de 1351...
Pegando página: 901 de 1351...
Pegando página: 951 de 1351...
Pegando página: 1001 de 1351...
Pegando página: 1051 de 1351...
Pegando página: 1101 de 1351...
Pegando página: 1151 de 1351...
Pegando página: 1201 de 1351...
Pegando página: 1251 de 1351...
Pegando página: 1301 de 1351...
Pegando página: 1351 de 1351...


Finalmente, carregamos os arquivos JSON e os combinamos em um único DataFrame do pandas.

In [9]:
df = pd.DataFrame(lista_desemb)
df

,nome,data_nascimento,municipio_nascimento,estado_nascimento,faculdade_direito,ano_formatura,data_magistratura,juiz_substituto_2grau,data_juiz_substituto_2grau,data_promocao_desembargador,data_aposentadoria,data_falecimento,quinto_constitucional,foi_promotor,page
0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1
1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2
2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,3
3,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,4
4,TRISTÃO DE ALENCAR ARARIPE,1821-10-07,Icó,Ceará,São Paulo,1845,NA,NA,NA,1871-10-06,1892-01-25,1908-07-03,NA,NA,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1346,RODOLFO PELLIZARI,1962-04-12,Araraquara,São Paulo,Faculdade de Direito de Sorocaba,1984,1990-08-24,sim,2017-02-16,2023-08-17,NA,NA,não,não,1347
1347,DANIELA MARIA CILENTO MORSELLO,1967-06-03,São Paulo,SP,Faculdade de Direito da Universidade de São Paulo,1989,1991-01-11,sim,2019-12-12,2023-09-21,NA,NA,não,não,1348
1348,EDUARDO VELHO NETO,1959-12-29,Bauru,São Paulo,Universidade Metodista de Piracicaba,1981,1985-03-22,NA,NA,2023-09-21,NA,NA,não,não,1349
1349,Jair de Souza,1963-11-17,São Caetano do Sul,São Paulo,Faculdade de Direito de São Bernardo do Campo,1986,1991-01-11,sim,2019-05-09,2023-09-21,NA,NA,não,não,1350


Agora, exportamos o DataFrame para um arquivo parquet, que pode ser usado para análise posterior.

In [10]:
df.to_parquet('parquet/desembargadores.parquet')

O próximo passo é realizar a limpeza da base, que fazemos no script `02-limpeza.r`, na linguagem R.